# Jupyter Notebook for phoneme coverage analysis

This jupyter notebook checks dataset configured in config.json for phoneme coverage.
As mentioned here https://github.com/mozilla/TTS/wiki/Dataset#what-makes-a-good-dataset a good phoneme coverage is recommended.

Most parameters will be taken from config.json file in mozilla tts repo so please ensure it's configured correctly for your dataset.
This notebook used lots of existring code from the TTS repo to ensure future compatibility.

Many thanks to Neil Stoker supporting me on this topic :-).

I provide this notebook without any warranty but it's hopefully useful for your dataset analysis.

Happy TTS'ing :-)

Thorsten Müller

* https://github.com/thorstenMueller/deep-learning-german-tts
* https://discourse.mozilla.org/t/contributing-my-german-voice-for-tts/

In [ ]:
import collections
import operator
from multiprocessing import Pool, cpu_count, set_start_method

from matplotlib import pylab as plt
from tqdm.notebook import tqdm

from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.text.tokenizer import TTSTokenizer

%matplotlib inline

set_start_method("fork", force=True)

In [ ]:
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.configs.shared_configs import BaseTTSConfig

dataset = BaseDatasetConfig(
    dataset_name="ljspeech",
    formatter="ljspeech",
    path="../../tests/data/ljspeech",
    meta_file_train="metadata.csv",
)
CONFIG = BaseTTSConfig(
    text_cleaner="phoneme_cleaners",
    phonemizer="espeak",
    phoneme_language="en-us",
    use_phonemes=True,
    datasets=[dataset],
)

CHARS_TO_REMOVE = ".,:!?'"

# Alternatively, you can load an existing config file:
# from TTS.config import load_config
# CONFIG = load_config("path/to/config.json")

In [ ]:
# Load some properties from config.json
CONFIG_METADATA = load_tts_samples(CONFIG.datasets, eval_split=False)[0]
RUN_NAME = CONFIG.run_name + " (" + CONFIG.run_description + ")"

# Needed to convert text to phonemes and phonemes to ids
tokenizer, config = TTSTokenizer.init_from_config(CONFIG)

In [ ]:
# print some debug information on loaded config values
print(" > Run name: " + RUN_NAME)
print(" > Dataset files: " + str(len(CONFIG_METADATA)))
print(" > Phoneme language: " + CONFIG.phoneme_language)
print(" > Phonemizer: " + CONFIG.phonemizer)
print(" > Used text cleaner: " + CONFIG.text_cleaner)
print(" > Enable eos bos chars: " + str(CONFIG.enable_eos_bos_chars))

In [ ]:
def get_phoneme_from_sequence(text):
    temp_list = []
    if len(text["text"]) > 0:
        temp_text = text["text"].rstrip("\n")
        for rm_bad_chars in CHARS_TO_REMOVE:
            temp_text = temp_text.replace(rm_bad_chars, "")
        seq = tokenizer.text_to_ids(temp_text)
        text = tokenizer.ids_to_text(seq)
        text = text.replace(" ", "")
        temp_list.append(text)
    return temp_list

In [ ]:
# Get phonemes from metadata
phonemes = []

with Pool(cpu_count() - 1) as p:
    phonemes = list(tqdm(p.imap(get_phoneme_from_sequence, CONFIG_METADATA), total=len(CONFIG_METADATA)))
    phonemes = [i for sub in phonemes for i in sub]

In [ ]:
s = ""
phonemeString = s.join(phonemes)

d = {}
collections._count_elements(d, phonemeString)
sorted_d = dict(sorted(d.items(), key=operator.itemgetter(1), reverse=True))

# remove useless keys
sorted_d.pop(" ", None)
sorted_d.pop("ˈ", None)

phonemesSum = len(phonemeString.replace(" ", ""))

print("Dataset contains " + str(len(sorted_d)) + " different ipa phonemes.")
print("Dataset consists of " + str(phonemesSum) + " phonemes")

In [ ]:
print("5 rarest phonemes")

rareList = dict(sorted(sorted_d.items(), key=operator.itemgetter(1), reverse=False)[:5])
for key, value in rareList.items():
    print(key + " --> " + str(value) + " occurrences")

In [ ]:
# create plot from analysis result

x = []
y = []

for key, value in sorted_d.items():
    x.append(key)
    y.append(value)

plt.figure(figsize=(50, 50))
plt.title("Phoneme coverage for " + RUN_NAME, fontsize=50)
plt.xticks(fontsize=50)
plt.yticks(fontsize=50)
plt.barh(x, y, align="center", alpha=1.0)
plt.gca().invert_yaxis()
plt.ylabel("phoneme", fontsize=50)
plt.xlabel("occurrences", fontsize=50)

for i, v in enumerate(y):
    plt.text(v + 2, i - 0.2, str(v), fontsize=20)
    plt.text(v + 2, i + 0.2, "(" + str(round(100 / phonemesSum * v, 2)) + "%)", fontsize=20)


plt.show()